## Cryptocurrency Unsupervised Learning

In [379]:
# Import Dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import hvplot.pandas

In [380]:
# Loading the dataset
file = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


#### Data Preprocessing Tasks

In [381]:
crypto_df.count()

Unnamed: 0         1252
CoinName           1252
Algorithm          1252
IsTrading          1252
ProofType          1252
TotalCoinsMined     744
TotalCoinSupply    1252
dtype: int64

In [382]:
# Removing all cryptocurrencies that are not trading
crypto_df = crypto_df[crypto_df.IsTrading == True]
crypto_df.count()

Unnamed: 0         1144
CoinName           1144
Algorithm          1144
IsTrading          1144
ProofType          1144
TotalCoinsMined     685
TotalCoinSupply    1144
dtype: int64

In [383]:
# Removing all cryptocurrencies that have no defined algorithm
crypto_df["Algorithm"].isnull().count()
# There are all defined
# But the following syntax will remove rows with non-null values in Algorithm column
# crypto_df = crypto_df[crypto_df.Algorithm.notnull()]

1144

In [384]:
# Removing the IsTrading column
crypto_df.drop('IsTrading', axis=1, inplace=True)

In [385]:
# Removing all cryptocurrencies with at least one null value
crypto_df = crypto_df.dropna()

In [386]:
# Removing all cryptocurrencies without coins mined
crypto_df= crypto_df[crypto_df.TotalCoinsMined != 0]
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [387]:
# Creating a DataFrame for all the names of cryptocurrencies
coins_name = pd.DataFrame(crypto_df, columns = ['Unnamed: 0','CoinName'])
# pcs_df.set_index('Unnamed: 0', inplace=True)
coins_name.set_index('Unnamed: 0', inplace=True)
coins_name.index.name=None
# coins_name = crypto_df.filter(['Unnamed: 0','CoinName'], axis=1)
# coins_name.set_index('Unnamed: 0')
coins_name.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [388]:
# Removing the CoinName column in the crypto_df
# drop_coin = crypto_df.drop('CoinName', axis=1)
crypto_df = pd.DataFrame(crypto_df, columns = ['Unnamed: 0','Algorithm','ProofType',
                                               'TotalCoinsMined', 'TotalCoinSupply'])
crypto_df.set_index('Unnamed: 0', inplace=True)
crypto_df.index.name=None
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [389]:
# Creating dummy variables for all of the text features
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [390]:
# Standardizing Data from X dataframe
crypto_scaled = StandardScaler().fit_transform(X)
print(crypto_scaled[0:101])


[[-0.11674788 -0.15286468 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [-0.09358885 -0.14499604 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [ 0.52587231  4.4937636  -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 ...
 [ 3.92312182  2.57553368 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [ 0.03483681 -0.00495791 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [-0.0769557  -0.12093438 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]]


### Reducing Data Dimensions Using PCA

In [391]:
from sklearn.decomposition import PCA

# Reducing to three principal components
pca = PCA(n_components=3)

In [392]:
# Applying Dimensionality Reduction
crypto_pca = pca.fit_transform(crypto_scaled)

In [393]:
# Transforming PCA data to a DataFrame
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"]
)
pcs_df.set_index(X.index, inplace=True)
pcs_df.index.name=None
pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.335286,0.921799,-0.465890
404,-0.318577,0.921666,-0.466455
1337,2.335877,1.454619,-0.609352
BTC,-0.139157,-1.277697,0.132061
ETH,-0.147730,-1.916692,0.290485
LTC,-0.171887,-1.095412,-0.006891
DASH,-0.382309,1.127301,-0.427347
XMR,-0.141461,-2.134477,0.354761
ETC,-0.146167,-1.916816,0.290446
ZEC,-0.148304,-1.914062,0.298958


### Clustering Using K-means

In [394]:
# Attaining the Explained Variance Ratio
pca.explained_variance_ratio_

array([0.02736965, 0.02083042, 0.02006618])

### Elbow Curve

In [395]:
# Finding the best value for K
inertia = []
k = list(range(1, 11))

# Calculating the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
# Creating Elbow Curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

### Function to Determine Best Value for K

In [396]:
def test_k_value(k, data):
    data=data.copy()
    model=KMeans(n_clusters=k, random_state=0)
    
    model.fit(data)
    predictions=model.predict(data)
    
    data["class"]=model.labels_
    
    return data

#### Testing K=4

In [397]:
four_clusters=test_k_value(4, pcs_df)
four_clusters.head()

,PC 1,PC 2,PC 3,class
42,-0.335286,0.921799,-0.465890,0
404,-0.318577,0.921666,-0.466455,0
1337,2.335877,1.454619,-0.609352,0
BTC,-0.139157,-1.277697,0.132061,1
ETH,-0.147730,-1.916692,0.290485,1


In [398]:
# Plotting the clusters on 3D plot for K=4
import plotly.express as px
fig=px.scatter_3d(
    four_clusters,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
    width=800,
)

fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Testing K=5

In [399]:
five_clusters=test_k_value(5, pcs_df)
five_clusters.head()

,PC 1,PC 2,PC 3,class
42,-0.335286,0.921799,-0.465890,1
404,-0.318577,0.921666,-0.466455,1
1337,2.335877,1.454619,-0.609352,4
BTC,-0.139157,-1.277697,0.132061,0
ETH,-0.147730,-1.916692,0.290485,0


In [400]:
# Plotting the clusters on 3D plot for K=5
fig=px.scatter_3d(
    five_clusters,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
    width=800,
)

fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Testing K=6

In [401]:
six_clusters=test_k_value(6, pcs_df)
six_clusters.head()

,PC 1,PC 2,PC 3,class
42,-0.335286,0.921799,-0.465890,0
404,-0.318577,0.921666,-0.466455,0
1337,2.335877,1.454619,-0.609352,5
BTC,-0.139157,-1.277697,0.132061,1
ETH,-0.147730,-1.916692,0.290485,1


In [402]:
# Plotting the clusters on 3D plot for K=6
fig=px.scatter_3d(
    six_clusters,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
    width=800,
)

fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Based on the above 3D plots for K=4, K=5 & K=6, it seems that K=4 is the best option.

In [403]:
# Creating DataFrame with results
df1 = pd.merge(crypto_df, four_clusters, left_index=True, right_index=True)
clustered_df = pd.merge(df1, coins_name, left_index=True, right_index=True)
clustered_df =clustered_df[['Algorithm','ProofType','TotalCoinsMined','TotalCoinSupply','PC 1',
                           'PC 2', 'PC 3','CoinName','class']]
clustered_df.rename(columns = {'class':'Class'}, inplace = True) 
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.335286,0.921799,-0.465890,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.318577,0.921666,-0.466455,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.335877,1.454619,-0.609352,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.139157,-1.277697,0.132061,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.147730,-1.916692,0.290485,Ethereum,1


### Visualizing Results

In [404]:
# 3D plot with parameters hover_name="CoinName" and hover_data="Algorithm"
fig=px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    color="Class",
    symbol="Class",
    width=800,
)

fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [405]:
# Table with All Current Tradable Cryptocurrencies
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply',
                 'TotalCoinsMined', 'Class'], width=600)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [406]:
# Plotting the clusters on 2D plot for x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class",
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)